# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2
import sys
import os
from PIL import Image
import platform

%matplotlib inline

current_OS = platform.system()
if current_OS == "Linux":
    print("Linux")
elif current_OS == "Windows":
    print("Windows")
else:
    print("But what?")
    exit(-1)
    


Windows


## Read in an Image

In [2]:
def read_image(image_file):
#reading in an image
    image = mpimg.imread(image_file)

    # Need for my imread() settings
    # image = (np.uint8)(image * 255)
    # print(image.dtype)

    print('This image is:', type(image), 'with dimensions:', image.shape)
    plt.imshow(image)  
    # if you wanted to show a single color channel image called 'gray',
    # for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def GetMaskByPoly(img, vertices):
    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    #masked_image = cv2.bitwise_and(img, mask)
    return mask
    
def GetMaskByColor(img, Color):
    """
    Applies an image mask.
    Only keeps the region of the image which is satisfied of color threshold.
    The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    
    color_thresholds = (img[:,:,0] < Color[0]) \
            | (img[:,:,1] < Color[1]) \
            | (img[:,:,2] < Color[2])
    mask = np.zeros_like(img) # + [255,255,255]
    mask[color_thresholds] = [0,0,0]
    mask[~color_thresholds] = [255,255,255]
    return mask


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [14]:
## Test Images

# Build your pipeline to work on the images in the directory "test_images"  
# **You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
def CheckImages():
    """
    Function checks all files in test_images/ and shows they as image
    """
    
    CurrentDir = "test_images/"
    names = os.listdir(CurrentDir)

    for filename in names:
        #FullFilename = CurrentDir+filename
        FullFilename = os.path.join(CurrentDir, filename)
        print(FullFilename)
        
        image = mpimg.imread(FullFilename)
        plt.imshow(image)
        plt.pause(1)


In [5]:
def ExtractTwoSides(line_segments, xcenter):
    '''
    Function tryes to group input line segments onto 2 clusters (L/R)
    and substitutes they to 2 single lines
    '''
    
    print(line_segments.dtype)
    #Select segments
    left_lines = np.empty_like(line_segments)
    right_lines = np.empty_like(line_segments)

    left_counter = 0
    right_counter = 0

    #left_lines = []
    #right_lines = []

    min_length = 4      #min segment len = 4 pixels
    min_length2 = min_length ** 2
    min_slope = 1/4     #minimal segment slope

#    imz = np.zeros((540,960,3),dtype=np.uint8)
#    draw_lines(imz, line_segments)
#    plt.imshow(imz)
#    plt.pause(1)

    counter = 0
    
    for line in line_segments:

        counter += 1
        line0 = line[0]
        
        dx = line0[2] - line0[0]
        dy = line0[3] - line0[1]

        #Reject lines with small slope
        if abs(dy)*3 < abs(dx):
            continue
        
        #orients segments from up to down
        if dy < 0:
            dy = -dy
            point1 = line0[:2].copy()
            line0[:2] = line0[2:]
            line0[2:] = point1
            #print('Exchanged: ',line0)


        length2 = dx**2 + dy**2
        if length2 < min_length2:
            continue

        length = math.sqrt(length2)
        if abs(dx)/length > min_slope:   #line almost vertical
            if line0[2] < xcenter:  #Left segment?
                left_lines[left_counter,0,:] = line0
                left_counter += 1
            else:
                right_lines[right_counter,0,:] = line0
                right_counter += 1
        else:
            if dx < 0:              #Left segment?
                left_lines[left_counter,:,:] = line0
                left_counter += 1
            elif dx > 0:            #Rightt segment?
                right_lines[right_counter,:,:] = line0
                right_counter += 1

    #Now there remains 2 groups of lines        
    left_lines.resize(left_counter,1,4)
    right_lines.resize(right_counter,1,4)

    n = left_lines.shape[0]
    if n:
        left_y1s = left_lines[:,0,1]
        left_top_point = left_y1s.argmin()
        left_y2s = left_lines[:,0,3]
        left_bott_point = left_y2s.argmax()

        left_seg = np.zeros((1,1,4), dtype=left_lines.dtype)
        left_seg[0,0,:2] = left_lines[left_top_point,0,:2]
        left_seg[0,0,2:] = left_lines[left_bott_point,0,2:]
        
        # averaging
#        lines_center_x = mean(left_lines[:,0,0] + left_lines[:,0,2]) / 2
#        lines_center_y = mean(left_lines[:,0,1] + left_lines[:,0,2]) / 2
#        lines_dx = (left_lines[:,0,2] - left_lines[:,0,0])
#        lines_dy = (left_lines[:,0,3] - left_lines[:,0,1])
#        mean_slope = mean(lines_dx / lines_dy)
        
    else:
        left_seg = np.array([], dtype=np.int32)

    n = right_lines.shape[0]
    if n:
        right_y1s = right_lines[:,0,1]
        right_top_point = right_y1s.argmin()
        right_y2s = right_lines[:,0,3]
        right_bott_point = right_y2s.argmax()

        right_seg = np.zeros((1,1,4), dtype=right_lines.dtype)
        right_seg[0,0,:2] = right_lines[right_top_point,0,:2]
        right_seg[0,0,2:] = right_lines[right_bott_point,0,2:]
    else:
        right_seg = np.array([], dtype=np.int32)

    return (left_seg, right_seg)


## Build a Lane Finding Pipeline
def FindLinesOnFrame(image, ShowFlag = False):
    """
    Function detects road lines on the picture and returns these segments
    """
    
    ysize = image.shape[0]
    xsize = image.shape[1]

    # ############ Color & Region selection
    color_select = np.copy(image)
    region_select = np.copy(image)
    
    #gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) #grayscale conversion
    gray = grayscale(image)

    #Color masking
    red_threshold = 120
    green_threshold = 120
    blue_threshold = 10
    rgb_threshold = [red_threshold, green_threshold, blue_threshold]

    ColorMask = GetMaskByColor(image, rgb_threshold)

    Ymid = round(ysize*0.6)
    A = [0, ysize-1]
    B = [round(xsize*0.5), Ymid]
    C = [round(xsize*0.55), Ymid]
    D = [xsize, ysize]

    ROI = np.array( [[A,B,C,D]], dtype=np.int32 )
    
    #ROI mask
    RegionMask = GetMaskByPoly(image, ROI)
    
    #ROI & Color mask
    ColorRegionMask = cv2.bitwise_and(RegionMask, ColorMask)
    
    MaskedImg = cv2.bitwise_and(ColorRegionMask, image)
    
    # Edges finding, given parameters from test Quiz
    kernel_size = 5
    low_threshold = 50
    high_threshold = 150

    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size), 0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    edges3 = np.dstack((edges, edges, edges))

    # Mask edges
    masked_edges = cv2.bitwise_and(ColorRegionMask, edges3)
    
    # Define the Hough transform parameters
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_length = 40
    max_line_gap = 20

    road_lines = cv2.HoughLinesP(masked_edges[:,:,0], rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    
# Extend lines segments
    xcenter = image.shape[1] // 2
    left_line, right_line = ExtractTwoSides(road_lines, xcenter)
    
#Extrapolate lines to ROI margins
    Ytop = ROI[0,1,1]
    Ybott = ROI[0,3,1]
    
    #The left line, if exists
    if left_line.shape[0]:
        x1 = left_line[0,0,0]
        y1 = left_line[0,0,1]

        x2 = left_line[0,0,2]
        y2 = left_line[0,0,3]
        
        dx = x2 - x1
        dy = y2 - y1
        
        if dx == 0:
            left_line[0,0,1] = Ytop
            left_line[0,0,3] = Ybott
        else:
            #calculate crosspoints with ROI

            #left_top
            dx1 = dx / dy * (y2 - Ytop)
            left_line[0,0,0] = x2 - dx1
            left_line[0,0,1] = Ytop

            #left_bott
            dx2 = dx / dy * (Ybott - y1)
            left_line[0,0,2] = x1 + dx2
            left_line[0,0,3] = Ybott

    #The right line, if exists
    if right_line.shape[0]:
        x1 = right_line[0,0,0]
        y1 = right_line[0,0,1]

        x2 = right_line[0,0,2]
        y2 = right_line[0,0,3]
        
        dx = x2 - x1
        dy = y2 - y1
        
        if dx == 0:
            right_line[0,0,1] = Ytop
            right_line[0,0,3] = Ybott
        else:
            #calculate crosspoints with ROI
            dx1 = dx / dy * (y2 - Ytop)
            right_line[0,0,0] = x2 - dx1
            right_line[0,0,1] = Ytop

            #right_bott
            dx2 = dx / dy * (Ybott - y1)
            right_line[0,0,2] = x1 + dx2
            right_line[0,0,3] = Ybott

    
    if ShowFlag:
    # Drawing the results
        # Draw the lines on the edge image
        line_image = np.copy(image)*0 #creating a blank to draw lines on
        draw_lines(line_image, left_line,[0,255,0])
        draw_lines(line_image, right_line,[0,0,255])

        combo = cv2.addWeighted(masked_edges, 0.8, line_image, 1, 0) 
    
        #1st plot
        #Original
        imageROI = np.copy(image)
        cv2.polylines(imageROI,ROI,True,[255,0,0],2)
    
        plt.subplot(2,2,1)
        plt.imshow(imageROI)
        plt.title('Original image')
        plt.tick_params(axis='both', labelsize=0, length = 0)

        plt.subplot(2,2,2)
        plt.imshow(RegionMask)
        plt.title('Region Mask')
        plt.tick_params(axis='both', labelsize=0, length = 0)

        #Color mask
        plt.subplot(2,2,3)
        plt.imshow(ColorMask)
        plt.title('Color Mask')
        plt.tick_params(axis='both', labelsize=0, length = 0)
    
        plt.subplot(2,2,4)
        plt.imshow(MaskedImg)
        plt.title('Color&Region Masked')
        tick_params(axis='both', labelsize=0, length = 0)
        plt.pause(3)

    #2nd plot
        plt.subplot(2,2,1)
        plt.imshow(image)
        plt.title('Original')
        plt.tick_params(axis='both', labelsize=0, length = 0)

    #Add ROI shape
        cv2.polylines(edges3,ROI,True,[255,0,0],10)
    
        plt.subplot(2,2,2)
        plt.imshow(edges3)
        plt.title('Raw edges')
        plt.tick_params(axis='both', labelsize=0, length = 0)

        plt.subplot(2,2,3)
        plt.imshow(masked_edges)
        plt.title('Masked edges')
        plt.tick_params(axis='both', labelsize=0, length = 0)

        plt.subplot(2,2,4)
        plt.imshow(combo)
        plt.title('Lines')
        plt.tick_params(axis='both', labelsize=0, length = 0)
    
    return left_line, right_line

Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [6]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

def overprint_lines(image, lines):

    line_image = np.copy(image)*0 #creating a blank to draw lines on
    draw_lines(line_image, lines)
    combo = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    return combo


def Process_Images():
    CurrentDir = "test_images"
    OutputDir = "test_images_output"

    names = os.listdir(CurrentDir)
    for filename in names:
        print(filename)

    if not os.path.exists(OutputDir):
        os.makedirs(OutputDir)

    for filename in names:
        FullFilename = os.path.join(CurrentDir, filename)
        print(FullFilename)
        image = mpimg.imread(FullFilename)

        left_line, right_line = FindLinesOnFrame(image)

        if left_line.shape[0] or right_line.shape[0]:
            if left_line.shape[0] and right_line.shape[0]:
                road_lines = np.append(left_line, right_line, axis=1)
            elif left_line.shape[0]:
                road_lines = left_line.copy
            else:
                road_lines = right_line.copy
            
            line_image = np.zeros_like(image)
            draw_lines(line_image, road_lines)

            # Draw the lines on the edge image
            combo = cv2.addWeighted(image, 0.8, line_image, 1, 0) 
        else:
            combo = image

        OutputName = os.path.join(OutputDir, filename)
        plt.imsave(OutputName, combo)

        plt.subplot(1,2,1)
        plt.imshow(image)
        plt.title(filename)
        plt.tick_params(axis='both', labelsize=0, length = 0)

        plt.subplot(1,2,2)
        plt.imshow(combo)
        plt.title('The same with Lines')
        plt.tick_params(axis='both', labelsize=0, length = 0)

        plt.pause(1)


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [10]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    left_line, right_line = FindLinesOnFrame(image)
    print('1 type(left_line): ', type(left_line))
    print('1 type(right_line): ', type(right_line))
    print('1 left_line.shape: ', left_line.shape)
    print('1 right_line.shape: ', right_line.shape)
    print('1 left_line.ndim: ', left_line.ndim)
    print('1 right_line.ndim: ', right_line.ndim)
    
    if left_line.shape[0] or right_line.shape[0]:
        if left_line.shape[0] and right_line.shape[0]:
            print('2 left_line.shape: ', left_line.shape)
            print('2 right_line.shape: ', right_line.shape)
            road_lines = np.append(left_line, right_line, axis=1)
            print('2 type(road_lines): ', type(road_lines))
        elif left_line.shape[0]:
            print('3 left_line.shape: ', left_line.shape)
            road_lines = left_line.copy()
        else:
            print('4 right_line.shape: ', right_line.shape)
            road_lines = right_line.copy()
            print('5 type(road_lines): ', type(road_lines))
            print('5 road_lines.shape: ', road_lines.shape)
            
        print('6 type(road_lines): ', type(road_lines))
        print('6 road_lines.shape: ', road_lines.shape)

        combo = overprint_lines(image, road_lines)
    else:
        combo = image
    
    return combo

Let's try the one with the solid white lane on the right first ...

In [11]:
def process_video(videofile):
    # Import everything needed to edit/save/watch video clips
    from moviepy.editor import VideoFileClip
    from IPython.display import HTML
    
    video_inpath = 'test_videos'
    video_outpath = 'test_videos_output'
    
    if not os.path.exists(video_outpath):
        os.mkdir(video_outpath)
    
    video_input = os.path.join(video_inpath, videofile)
    if not os.path.exists(video_input):
        print("ERROR!!!")
        return
    
    video_output = os.path.join(video_outpath, videofile)
    if os.path.exists(video_output):
        os.remove(video_output)
        
    print("Input video: " + video_input)
    print("Output video: " + video_output)

    ## To speed up the testing process you may want to try your pipeline 
    ## on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values 
    ## representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
    
    video1 = VideoFileClip(video_input)
#    This lines have to been added in case of videowriter incorrect behavior
#    video1.reader.close()
#    video1.audio.reader.close_proc()
    
    video2 = video1.fl_image(process_image) #NOTE: this function expects color images!!
    
    %time video2.write_videofile(video_output, audio=False)
    #video2.write_videofile(video_output, audio=False)
    
    global current_OS
    
    if current_OS == "Linux":
        HTML("""
        <video width="960" height="540" controls>
        <source src="{0}">
        </video>
        """.format(white_output))
    
    print('Done!')

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

Now for the one with the solid yellow lane on the left. This one's more tricky!

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


In [12]:
#Process_Images()

process_video('solidWhiteRight.mp4')
#process_video('solidYellowLeft.mp4')
#process_video('challenge.mp4')


Input video: test_videos\solidWhiteRight.mp4
Output video: test_videos_output\solidWhiteRight.mp4
int32
1 type(left_line):  <class 'numpy.ndarray'>
1 type(right_line):  <class 'numpy.ndarray'>
1 left_line.shape:  (1, 1, 4)
1 right_line.shape:  (1, 1, 4)
1 left_line.ndim:  3
1 right_line.ndim:  3
2 left_line.shape:  (1, 1, 4)
2 right_line.shape:  (1, 1, 4)
2 type(road_lines):  <class 'numpy.ndarray'>
6 type(road_lines):  <class 'numpy.ndarray'>
6 road_lines.shape:  (1, 2, 4)


Moviepy - Building video test_videos_output\solidWhiteRight.mp4.


Moviepy - Writing video test_videos_output\solidWhiteRight.mp4



t:   0%|                                     | 0/221 [00:00<?, ?it/s, now=None]

int32
1 type(left_line):  <class 'numpy.ndarray'>
1 type(right_line):  <class 'numpy.ndarray'>
1 left_line.shape:  (1, 1, 4)
1 right_line.shape:  (1, 1, 4)
1 left_line.ndim:  3
1 right_line.ndim:  3
2 left_line.shape:  (1, 1, 4)
2 right_line.shape:  (1, 1, 4)
2 type(road_lines):  <class 'numpy.ndarray'>
6 type(road_lines):  <class 'numpy.ndarray'>
6 road_lines.shape:  (1, 2, 4)
int32
1 type(left_line):  <class 'numpy.ndarray'>
1 type(right_line):  <class 'numpy.ndarray'>
1 left_line.shape:  (1, 1, 4)
1 right_line.shape:  (1, 1, 4)
1 left_line.ndim:  3
1 right_line.ndim:  3
2 left_line.shape:  (1, 1, 4)
2 right_line.shape:  (1, 1, 4)
2 type(road_lines):  <class 'numpy.ndarray'>
6 type(road_lines):  <class 'numpy.ndarray'>
6 road_lines.shape:  (1, 2, 4)


...

int32
1 type(left_line):  <class 'numpy.ndarray'>
1 type(right_line):  <class 'numpy.ndarray'>
1 left_line.shape:  (1, 1, 4)
1 right_line.shape:  (1, 1, 4)
1 left_line.ndim:  3
1 right_line.ndim:  3
2 left_line.shape:  (1, 1, 4)
2 right_line.shape:  (1, 1, 4)
2 type(road_lines):  <class 'numpy.ndarray'>
6 type(road_lines):  <class 'numpy.ndarray'>
6 road_lines.shape:  (1, 2, 4)


Moviepy - Done !


Moviepy - video ready test_videos_output\solidWhiteRight.mp4


t:   3%|▉                            | 7/221 [02:07<00:21,  9.80it/s, now=None]

Wall time: 20.6 s
Done!
